# Preface

This first exercise is primarily meant to ensure that everything is working fine. Complete the assignment by filling in code where necessary using Jupyter in Colab.

# Numpy tutorial
You'll need to know some numpy to work with vectors. If you are not familiar to numpy, we recommend to check the following Python/Numpy tutorial:

   http://cs231n.github.io/python-numpy-tutorial/

We recommend to follow it up to scipy section at least, but learning Matplotlib might be helpful plots learning curves and similar.

# Making sure everything works

If you are planning to run the labs in your laptop, you'll need to make sure you have Python, Jupyter, and TensorFlow installed. For some help with that, see 'Getting set up' document in  [Egela](https://egela1718.ehu.eus/mod/resource/view.php?id=735186). Please, make all the installations outside the class. 

Once you've done all of that, you should open this notebook in Jupyter and run the below:

In [0]:
import numpy as np

If that worked as expected, you should be able to run the below a few times and get different outcomes each time.

In [2]:
np.random.rand()

0.535757587543366

Now let's try importing and testing TensorFlow. This can be a bit trickier to install properly. Even once it's installed, running this line should take a few seconds.

In [3]:
%tensorflow_version 2.x
import tensorflow as tf
tf.__version__

TensorFlow 2.x selected.


'2.1.0-rc1'

First we define some tensor variables.

In [0]:
random_scalar = tf.random.uniform(())

Then we call a tensorflow function to get its value:

In [5]:
tf.print(random_scalar)


0.423511624


Variables can depend on other variables...

In [0]:
double_random_scalar = 2 * random_scalar
double_random_scalar_gt_one = double_random_scalar > 1

In [9]:
tf.print(double_random_scalar_gt_one)

0


Both TensorFlow and NumPy allow nearly any variable to take the form of a tensor (i.e., a vector, a matrice, or a higher-order such structure):

In [10]:
np.random.rand(2,3)

array([[0.92961522, 0.56997089, 0.27295463],
       [0.84833766, 0.53025155, 0.68255973]])

In [11]:
random_tensor = tf.random.uniform((2,3))
double_random_tensor = 2 * random_tensor
double_random_tensor_gt_one = double_random_tensor > 1
tf.print(random_tensor)
tf.print(double_random_tensor_gt_one)


[[0.21218133 0.999893665 0.57618]
 [0.573676467 0.767962217 0.360200167]]
[[0 1 1]
 [1 1 0]]


## 1. Loading the data

Let's load the Stanford Sentiment Treebank. The data can be originaly downloaded from here: [the train/dev/test Stanford Sentiment Treebank distribution](http://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip). If you already copied 2019-2020_labs folder to your Colab Notebooks``, you should have the data for this lab 2019-2020_labs/data/trees`.

In order to load the data, yiu we'll need to mount your Drive folder first and give the access to the Notebook. This will require one-step authentication. Please when you run the cell below follow the instructions.

Once you mount everything, make sure sst_home = 'drive/My Drive/Colab Notebooks/2019-2020_labs/data/trees/'' is correct path for the data.

Please run the cell below to upload the following data files:

    2019-2020_labs/data/trees/train.txt.
    2019-2020_labs/data/trees/dev.txt.
    2019-2020_labs/data/trees/test.txt.


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# set seed for replicability of results
import numpy as np
import tensorflow as tf

np.random.seed(1)
tf.random.set_seed(2)

In [15]:
# Load the data
import re

# Let's do 2-way positive/negative classification instead of 5-way    
def load_sst_data(path,
                  easy_label_map={0:0, 1:0, 2:None, 3:1, 4:1}):
    data = []
    with open(path) as f:
        for i, line in enumerate(f): 
            example = {}
            example['label'] = easy_label_map[int(line[1])]
            if example['label'] is None:
                continue
            
            # Strip out the parse information and the phrase labels---we don't need those here
            text = re.sub(r'\s*(\(\d)|(\))\s*', '', line)
            example['text'] = text[1:]
            data.append(example)
    return data

sst_home = 'drive/My Drive/2019-2020_labs/data/trees/'
training_set = load_sst_data(sst_home + 'train.txt')
dev_set = load_sst_data(sst_home + 'dev.txt')
test_set = load_sst_data(sst_home + 'test.txt')

print('Training size: {}'.format(len(training_set)))
print('Dev size: {}'.format(len(dev_set)))
print('Test size: {}'.format(len(test_set)))

Training size: 6920
Dev size: 872
Test size: 1821


## 2. Examining the data

In [16]:
# Print a sample of negative text chunks
[example["text"] for example in training_set if example["label"] == 0][:10]

["This is n't a new idea .",
 "... a sour little movie at its core ; an exploration of the emptiness that underlay the relentless gaiety of the 1920 's ... The film 's ending has a `` What was it all for ? ''",
 'Made me unintentionally famous -- as the queasy-stomached critic who staggered from the theater and blacked out in the lobby .',
 'The modern-day royals have nothing on these guys when it comes to scandals .',
 "It 's only in fairy tales that princesses that are married for political reason live happily ever after .",
 'An absurdist spider web .',
 'By no means a slam-dunk and sure to ultimately disappoint the action fans who will be moved to the edge of their seats by the dynamic first act , it still comes off as a touching , transcendent love story .',
 "It 's not a great monster movie .",
 "Too often , Son of the Bride becomes an exercise in trying to predict when a preordained `` big moment '' will occur and not `` if . ''",
 'A party-hearty teen flick that scalds like aci

In [17]:
# Print a sample of positive text chunks
[example["text"] for example in training_set if example["label"] == 1][:10]

["The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .",
 "The gorgeously elaborate continuation of `` The Lord of the Rings '' trilogy is so huge that a column of words can not adequately describe co-writer\\/director Peter Jackson 's expanded vision of J.R.R. Tolkien 's Middle-earth .",
 'Singer\\/composer Bryan Adams contributes a slew of songs -- a few potential hits , a few more simply intrusive to the story -- but the whole package certainly captures the intended , er , spirit of the piece .',
 'Yet the act is still charming here .',
 "Whether or not you 're enlightened by any of Derrida 's lectures on `` the other '' and `` the self , '' Derrida is an undeniably fascinating and playful fellow .",
 'Just the labour involved in creating the layered richness of the imagery in this chiaroscuro of madness and light is astonishing .',
 'Part of the charm of 

## Assignments
### Part 1:

Write a python function using NumPy to compute the following function of `x`. You can set $\mu$ to 0 and $\sigma$ to 1. This happens to be the probability distribution function for a normal distribution, but we're just using it as an arbitrary demo, and you shouldn't use any preexisting code for this particular distribution. You'll likely need to search for relevant NumPy documentation.

![The PDF of the standard normal distribution.](https://drive.google.com/uc?id=11NpGnvDTRhnEkFwDsRYdLKMYDGHUMODM)



In [0]:
def np_fn(x):
  mu = 0
  sigma = 1
  upper = pow((x- mu),2)
  down = 2*pow(sigma,2)
  result = (1 / np.sqrt(down * np.pi)) * (np.exp (-(upper / down)))
  return result

Assume `x` is a vector. You should be able to run the following command and get the subsequent result:

In [129]:
x = np.array([0, 1, 2, 3])
np_fn(x)

array([0.39894228, 0.24197072, 0.05399097, 0.00443185])

Expected output: `array([ 0.39894228,  0.24197072,  0.05399097,  0.00443185])
`

### Part 2:
Now try to write the same function (`tf_fn(x)`) in TensorFlow.

In [0]:
def tf_fn(x):
  mu = tf.constant(0.0,dtype='float32')
  sigma = tf.constant(1.0,dtype='float32')
  result = (1 / tf.math.sqrt(2*sigma**2 * np.pi)) * (np.exp (-((x- mu)**2 / 2*sigma**2)))
  return result

You should be able to run command below, and get the same output as above.

In [149]:
tf.print(tf_fn(x))

[0.398942292 0.241970733 0.0539909676 0.00443184841]


# Atribution:
Adapted by Oier Lopez de Lacalle and Olatz Perez de Viñaspre, based on a notebook by Sam Bowman at NYU